# Structured Output with Agents

<h3> In this tutorial we see how we can use the structured Outputs, along with agents </h3>
<h3>Structured tools in Microsoft Agent-Framework are used to ensure that agent responses follow a predictable schema (like JSON or Pydantic models), making outputs machine-readable, reliable, and easy to integrate into downstream applications. They transform free-form text into structured data that can be programmatically consumed.</h3>

In [84]:
# Installation refer 'basic_food_agent.ipynb' , installation section
# !pip install -U agent-framework --pre
# You can see the basic_food_agent.ipynb for basic agent run

In [85]:
# Import Required Libraries
import os
import random
# load dotenv library
from dotenv import load_dotenv, find_dotenv

In [91]:
load_dotenv(find_dotenv())

True

In [92]:
# Import Microsoft Agent Framework components
# ChatAgent : The main Agent class of the Conversational AI
# OpenAIChatClient - Client for loading llm_inference, here we will use OPENROUTER instead of OPENAI
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

In [93]:
import requests
import json
from typing import Annotated, List, Dict, Any, Optional
from pydantic import Field

# Clean the MealDB response for the LLM
def _clean_meal_data(meal: Dict[str, Any]) -> Dict[str, Any]:
    """
    Helper function to restructure the raw meal API response into a clean, 
    LLM-friendly format by combining ingredients and measures.
    """
    if not meal:
        return {}

    # Combine ingredients and measures into a single list
    ingredients = []
    for i in range(1, 21):
        ing = meal.get(f"strIngredient{i}")
        measure = meal.get(f"strMeasure{i}")
        if ing and ing.strip():
            ingredients.append(f"{measure.strip()} {ing.strip()}".strip())

    return {
        "id": meal.get("idMeal"),
        "name": meal.get("strMeal"),
        "category": meal.get("strCategory"),
        "area": meal.get("strArea"),
        "instructions": meal.get("strInstructions"),
        "ingredients": ingredients,
        "tags": meal.get("strTags"),
        "youtube_link": meal.get("strYoutube")
    }

# Get random meal for today
def get_random_meal() -> str:
    """
    Retrieves a random meal recipe from the database. 
    Useful when the user wants a surprise suggestion or explicitly asks for a random recommendation.

    Returns:
        str: A JSON string containing the meal name, ingredients, and cooking instructions.
    """
    try:
        response = requests.get("https://www.themealdb.com/api/json/v1/1/random.php")
        response.raise_for_status()
        data = response.json()
        
        if not data.get("meals"):
            return json.dumps({"error": "No meal found."})
            
        # meal = _clean_meal_data(data["meals"][0])
        # return json.dumps(meal, indent=2)
        return data
        
    except Exception as e:
        return json.dumps({"error": f"Failed to fetch random meal: {str(e)}"})

# Here we get the meal by name
def get_meal_by_name(
    meal_name: Annotated[str, Field(description="The name of the meal to search for (e.g., 'Arrabiata', 'Burger').")]
) -> str:
    """
    Searches for a specific meal recipe by name. 
    Use this when the user asks for a specific dish or wants to know how to cook a named item.

    Args:
        meal_name: The name of the dish to search for.

    Returns:
        str: A JSON string containing a list of matching meals with their details.
    """
    try:
        # The API requires a search query parameter 's'
        response = requests.get(f"https://www.themealdb.com/api/json/v1/1/search.php?s={meal_name}")
        response.raise_for_status()
        data = response.json()
        
        if not data.get("meals"):
            return json.dumps({"status": "not_found", "message": f"No meals found with the name '{meal_name}'."})
        
        # # Clean and limit results (e.g., top 3 matches to save tokens)
        # results = [_clean_meal_data(m) for m in data["meals"][:3]]
        # return json.dumps(results, indent=2)
        return data

    except Exception as e:
        return json.dumps({"error": f"Failed to search for meal: {str(e)}"})

<h3> Here Let's make the Food Details in Structured Format, if you see, I have commented out the clean_meal function from get_random_meal and get_meal_by_name </h3>
<h3> Our Agent would be able to get the Meal data in structured format.</h3>

In [94]:
# Adding structured Output here
from pydantic import BaseModel

class MealInfo(BaseModel):
    """Information about a Meal"""
    id: str | None = None
    name: str | None = None
    category: str | None = None
    area: str | None = None
    instructions: str | None = None
    ingredients: List[str] | None = None
    tags: str | None = None
    youtube_link: str | None = None

In [95]:
# Setup OpenAIChatClient for LLM Inference - Here we will use OpenRouter API which is compatible with OpenAI and NVIDIA 30B model
# This client connects to the OpenRouter Models which are OpenAI-compatible endpoint
# Environment variables required
# OPENROUTER_ENDPOINT - 
# OPENROUTER_API_KEY
openai_chat_client = OpenAIChatClient(
    base_url=os.environ.get("GROQ_ENDPOINT"),
    api_key=os.environ.get("GROQ_API_KEY"),
    model_id="openai/gpt-oss-20b"
)

In [96]:
AGENT_NAME = "FoodAgent"

AGENT_INSTRUCTIONS = """You are an expert AI Chef dedicated to helping users discover and prepare delicious meals.

CORE BEHAVIORS:
1. **Tool Usage**: You have access to a recipe database. ALWAYS use the provided tools to answer questions about recipes. Do not guess or hallucinate ingredients.
   - Use `get_meal_by_name` when the user asks for a specific dish.
   - Use `get_random_meal` when the user is undecided, asks for a suggestion, or wants a surprise.

2. **Response Format**: 
   - Extract the Meal Information from the tool results
   - You MUST return your response as a valid JSON object matching the MealInfo schema
   - The JSON must include these fields: id, name, category, area, instructions, ingredients (as an array), tags, youtube_link
   - Combine the strIngredient and strMeasure fields from the API response into a single \"measure ingredient\" string for each ingredient

3. **Constraints**: 
   - Keep your response strictly as JSON only - no additional text, markdown formatting, or explanations
   - Your entire response should be a single valid JSON object
   - Do not include any text before or after the JSON

4. **MealInfo Schema**:
   - id: meal ID (string or null)
   - name: meal name (string or null)
   - category: meal category (string or null)
   - area: origin area (string or null)
   - instructions: cooking instructions (string or null)
   - ingredients: array of strings combining measure and ingredient
   - tags: meal tags (string or null)
   - youtube_link: youtube URL (string or null)

Example format:
{
  \"id\": \"53344\",
  \"name\": \"Jamaican Sweet Potato Pudding\",
  \"category\": \"Dessert\",
  \"area\": \"Jamaican\",
  \"instructions\": \"Preheat oven to...\",
  \"ingredients\": [\"2 Lbs Sweet Potatoes\", \"1 1/2 cup Coconut Milk\"],
  \"tags\": \"Sweet,Spicy\",
  \"youtube_link\": \"https://www.youtube.com/watch?v=...\"
}
"""

In [97]:
# Here we add the logging Function middleware since we are working with functions
from agent_framework import AgentRunContext, FunctionInvocationContext
from typing import Callable , Awaitable

# Function middleware 
async def logging_function_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """Middleware that logs function calls."""
    print(f"Calling function: {context.function.name}")

    await next(context)

    print(f"Function result: {context.result}")


In [98]:
# create the agent remember we are not using any tools here, this is simple example
# T
agent = ChatAgent(
    name = AGENT_NAME,
    chat_client=openai_chat_client,
    instructions=AGENT_INSTRUCTIONS,
    tools=[get_random_meal, get_meal_by_name],
    middleware=[logging_function_middleware]
)

<h2> Run the agent and parse the structured JSON output manually</h2>
<h3> Since the NVIDIA model doesn't support response_format parameter, we parse the JSON from response.text and convert to MealInfo</h3>

In [99]:
# Let's get the response - Note: response_format may not work with NVIDIA model via OpenRouter
response = await agent.run("What should I eat today?", response_format=MealInfo)

Calling function: get_random_meal
Function result: {'meals': [{'idMeal': '53303', 'strMeal': 'Raspberry mousse', 'strMealAlternate': None, 'strCategory': 'Dessert', 'strArea': 'Polish', 'strInstructions': 'step 1\r\nPut the gelatine leaves in a bowl, cover with warm water and leave to soak for 5 mins. Drain and squeeze out any excess water.\r\n\r\nstep 2\r\nTip the raspberries into a pan over a medium-low heat along with the sugar and lemon juice. Cook for 5-6 mins until the berries have completely broken down. Push the mixture through a sieve set over a bowl, discarding the seeds. Stir in the gelatine leaves until dissolved (if they don’t dissolve, pour the mixture into a clean saucepan and heat gently until dissolved). Set aside to cool for 15 mins.\r\n\r\nstep 3\r\nWhip the cream to soft peaks using an electric whisk, then gently fold this into the raspberry mixture. Spoon into the ramekins or moulds and chill overnight, or for at least 6 hrs. Serve with raspberries scattered over t

In [100]:
# Parse JSON from response.text and convert to MealInfo
import re

def extract_json(text: str) -> dict:
    """Extract JSON object from text, handling common formatting issues."""
    # Try to find JSON object using regex
    json_pattern = r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}'
    matches = re.findall(json_pattern, text)
    
    for match in matches:
        try:
            return json.loads(match)
        except json.JSONDecodeError:
            continue
    
    # If no match found, try parsing the whole text
    try:
        return json.loads(text.strip())
    except json.JSONDecodeError:
        raise ValueError("No valid JSON found in response")

# Extract and parse the JSON
meal_data = extract_json(response.text)
meal_info = MealInfo(**meal_data)

print(f"Meal Info: {meal_info}")
print(f"\nMeal Name: {meal_info.name}")
print(f"Category: {meal_info.category}")
print(f"Ingredients: {meal_info.ingredients}")

Meal Info: id='53303' name='Raspberry mousse' category='Dessert' area='Polish' instructions='step 1\r\nPut the gelatine leaves in a bowl, cover with warm water and leave to soak for 5 mins. Drain and squeeze out any excess water.\r\n\r\nstep 2\r\nTip the raspberries into a pan over a medium-low heat along with the sugar and lemon juice. Cook for 5-6 mins until the berries have completely broken down. Push the mixture through a sieve set over a bowl, discarding the seeds. Stir in the gelatine leaves until dissolved (if they don’t dissolve, pour the mixture into a clean saucepan and heat gently until dissolved). Set aside to cool for 15 mins.\r\n\r\nstep 3\r\nWhip the cream to soft peaks using an electric whisk, then gently fold this into the raspberry mixture. Spoon into the ramekins or moulds and chill overnight, or for at least 6 hrs. Serve with raspberries scattered over the top.' ingredients=['2 Gelatine Leafs', '350g Raspberries', '60g Caster Sugar', 'Juice of 1 Lemon', '500ml Whip

In [104]:
if response.value:
    meal_info = response.value
    print(f"Name: {person_info.name}, Age: {person_info.age}, Occupation: {person_info.occupation}")
else:
    print("No structured data found in response")

No structured data found in response


In [105]:
response.value

In [106]:
response.text

'{"id":"53303","name":"Raspberry mousse","category":"Dessert","area":"Polish","instructions":"step 1\\r\\nPut the gelatine leaves in a bowl, cover with warm water and leave to soak for 5 mins. Drain and squeeze out any excess water.\\r\\n\\r\\nstep 2\\r\\nTip the raspberries into a pan over a medium-low heat along with the sugar and lemon juice. Cook for 5-6 mins until the berries have completely broken down. Push the mixture through a sieve set over a bowl, discarding the seeds. Stir in the gelatine leaves until dissolved (if they don’t dissolve, pour the mixture into a clean saucepan and heat gently until dissolved). Set aside to cool for 15 mins.\\r\\n\\r\\nstep 3\\r\\nWhip the cream to soft peaks using an electric whisk, then gently fold this into the raspberry mixture. Spoon into the ramekins or moulds and chill overnight, or for at least 6 hrs. Serve with raspberries scattered over the top.","ingredients":["2 Gelatine Leafs","350g Raspberries","60g Caster Sugar","Juice of 1 Lemon